In [1]:
# original code
import torch
from tqdm import tqdm
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv,GATConv
from torch_geometric.utils import train_test_split_edges
from torch_geometric.nn import VGAE

import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "0"

def decode(model,epochs,out_channels,data,device):
    num_features = data.num_node_features
    x = data.x.to(device)
    edge_index = data.edge_index.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    model.train()
    loss = 0
    for epoch in tqdm(range(1, epochs + 1)): # train
        all_loss = []
        optimizer.zero_grad()
        z = model.encode(x, edge_index)
        recon_loss = model.recon_loss(z, edge_index) # how to calculate the loss
        loss = recon_loss +20* (1 / data.num_nodes) * model.kl_loss()  # new line
        #print("all loss is ",float(loss))
        all_loss.append(loss)
        loss.backward()
        if(epoch%10==0):
            print("epoch is {} and loss is {}".format(epoch,loss))
        optimizer.step()
    model.eval()
    with torch.no_grad(): # pred
        z= model.encode(x, edge_index)
        print("z",z)
        print("z.shape",z.shape) # torch.Size([42372, 128])
    return z

class VGAE_gat(torch.nn.Module):
    def __init__(self, in_channels, out_channels,heads):
        super(VGAE_gat, self).__init__()      
        self.conv1 = GATConv(in_channels, out_channels,heads[0]) # cached only for transductive learning
        self.conv2 = GATConv(heads[0]*out_channels, out_channels,heads[1]) # cached only for transductive learning
        self.conv_mu = GATConv(heads[1]* out_channels, out_channels,heads[2],concat = False)
        self.conv_logstd = GATConv(heads[1]* out_channels, out_channels, heads[2],concat =False)
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)

# 数据为边和每条边的特征
with open("./transitions.txt") as f:
    edges1 = [list(map(int,[line.split()[0],line.split()[1]])) for line in f.readlines()]
    # edges2 = [list(map(int,[line.split()[1],line.split()[0]])) for line in f.readlines()]
    # edges = edges1+edges2
    edges = edges1
    # print(len(edges))
    # print((edges1))

edges = torch.tensor(edges,dtype=torch.long)
print(edges.shape)
edges = edges.transpose(0,1)
print(edges.shape) 

from torch_geometric.data import Data
import numpy as np

lac_features = np.load('./lac_feature.npy')


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')

out_dims = 64
data = Data(x= torch.tensor(lac_features,dtype=torch.float),edge_index=edges) # 修改其中的features

num_features = data.num_node_features

model = VGAE(VGAE_gat(num_features, out_dims,[2,2,1]))  # new line 修改其中的num_features
model = model.to(device)
z_gat= decode(model,100,out_dims,data,device) # 修改其中的data

torch.Size([603136, 2])
torch.Size([2, 603136])


 10%|█         | 10/100 [00:08<01:21,  1.10it/s]

epoch is 10 and loss is 2.00590181350708


 20%|██        | 20/100 [00:17<01:09,  1.15it/s]

epoch is 20 and loss is 1.5881938934326172


...

epoch is 80 and loss is 1.402797818183899


 90%|█████████ | 90/100 [01:13<00:07,  1.26it/s]

epoch is 90 and loss is 1.3994593620300293


100%|██████████| 100/100 [01:21<00:00,  1.22it/s]

epoch is 100 and loss is 1.397560954093933
z tensor([[ 0.3915, -0.1769, -0.0286,  ..., -0.2984,  0.0428, -0.5700],
        [ 0.3586, -0.1540, -0.0115,  ..., -0.2894,  0.0317, -0.5544],
        [ 0.4441, -0.1988, -0.0242,  ..., -0.3523,  0.0427, -0.6565],
        ...,
        [-0.1110, -0.0858, -0.0069,  ...,  0.0252,  0.0074,  0.1053],
        [ 0.0198, -0.0317,  0.0058,  ...,  0.0969, -0.0150,  0.0019],
        [-0.0203,  0.0250,  0.0243,  ...,  0.0689,  0.0606,  0.1998]],
       device='cuda:0')
z.shape torch.Size([6053, 64])


In [3]:
# 打开一个文本文件以写入模式
with open(f'/root/autodl-tmp/data/operator0/graph/road_embed_vgae.txt', 'w') as file:
    first_line = f"{z_gat.shape[0]} {z_gat.shape[1]}\n"
    file.write(first_line)

    # 遍历每一行，对每行元素求和后写入
    for index, row in enumerate(z_gat):
        row_value = " ".join(str(value.item()) for value in row)
        line = f"{index} {row_value}\n"
        file.write(line)

In [26]:
# original code
import torch
from tqdm import tqdm
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv,GATConv
from torch_geometric.utils import train_test_split_edges
from torch_geometric.nn import VGAE

import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

def decode(model,epochs,out_channels,data,device):
    num_features = data.num_node_features
    x = data.x.to(device)
    edge_index = data.edge_index.to(device)
    print(f"x device: {x.device}, edge_index device: {edge_index.device}")  # 添加打印设备信息
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    model.train()
    loss = 0
    for epoch in tqdm(range(1, epochs + 1)): # train
        all_loss = []
        optimizer.zero_grad()
        print(f"Before model.encode, device: {device}")  # 添加打印语句
        z = model.encode(x, edge_index)
        print(f"After model.encode, z device: {z.device}")  # 添加打印语句
        recon_loss = model.recon_loss(z, edge_index) # how to calculate the loss
        loss = recon_loss +20* (1 / data.num_nodes) * model.kl_loss()  # new line
        #print("all loss is ",float(loss))
        all_loss.append(loss)
        loss.backward()
        if(epoch%10==0):
            print("epoch is {} and loss is {}".format(epoch,loss))
        optimizer.step()
    model.eval()
    with torch.no_grad(): # pred
        print(f"Before eval model.encode, device: {device}")  # 添加打印语句
        z= model.encode(x, edge_index)
        print("z",z)
        print("z.shape",z.shape) # torch.Size([42372, 128])
        print(f"After eval model.encode, z device: {z.device}")  # 添加打印语句
    return z

class VGAE_gat(torch.nn.Module):
    def __init__(self, in_channels, out_channels,heads):
        super(VGAE_gat, self).__init__()      
        self.conv1 = GATConv(in_channels, out_channels,heads[0]) # cached only for transductive learning
        self.conv2 = GATConv(heads[0]*out_channels, out_channels,heads[1]) # cached only for transductive learning
        self.conv_mu = GATConv(heads[1]* out_channels, out_channels,heads[2],concat = False)
        self.conv_logstd = GATConv(heads[1]* out_channels, out_channels, heads[2],concat =False)
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)

# 数据为边和每条边的特征
with open("./transitions.txt") as f:
    edges1 = [list(map(int,[line.split()[0],line.split()[1]])) for line in f.readlines()]
    edges2 = [list(map(int,[line.split()[1],line.split()[0]])) for line in f.readlines()]
    edges = edges1+edges2
    # print(len(edges))
    # print((edges1))

edges = torch.tensor(edges,dtype=torch.long)
print(edges.shape)
edges = edges.transpose(0,1)
print(edges.shape) 

from torch_geometric.data import Data
import numpy as np

lac_features = np.load('./lac_feature.npy')


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')

out_dims = 64
data = Data(x= torch.tensor(lac_features,dtype=torch.float),edge_index=edges) # 修改其中的features

num_features = data.num_node_features

model = VGAE(VGAE_gat(num_features, out_dims,[2,2,1]))  # new line 修改其中的num_features
model = model.to(device)
z_gat= decode(model,100,out_dims,data,device) # 修改其中的data
    
np.save('./embedding_npy2txt/embedding', z_gat)

torch.Size([603136, 2])
torch.Size([2, 603136])
x device: cuda:0, edge_index device: cuda:0


  0%|          | 0/100 [00:00<?, ?it/s]

Before model.encode, device: cuda


RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript (most recent call last):
RuntimeError: nvrtc: error: invalid value for --gpu-architecture (-arch)

nvrtc compilation failed: 

#define NAN __int_as_float(0x7fffffff)
#define POS_INFINITY __int_as_float(0x7f800000)
#define NEG_INFINITY __int_as_float(0xff800000)


template<typename T>
__device__ T maximum(T a, T b) {
  return isnan(a) ? a : (a > b ? a : b);
}

template<typename T>
__device__ T minimum(T a, T b) {
  return isnan(a) ? a : (a < b ? a : b);
}

extern "C" __global__
void fused_sub_exp(float* tsrc_1, float* tsrc_max_9, float* aten_exp) {
{
if ((long long)(threadIdx.x) + 512ll * (long long)(blockIdx.x)<1213558ll ? 1 : 0) {
    float v = __ldg(tsrc_1 + (long long)(threadIdx.x) + 512ll * (long long)(blockIdx.x));
    float v_1 = __ldg(tsrc_max_9 + (long long)(threadIdx.x) + 512ll * (long long)(blockIdx.x));
    aten_exp[(long long)(threadIdx.x) + 512ll * (long long)(blockIdx.x)]